In [2]:
import pandas as pd 
import numpy as np 
import time
import geopandas as gpd
import plotly.express as px
import matplotlib.pyplot as plt
import plotly.graph_objects as go

In [ ]:
# outdated df 
df = pd.read_feather('pidgeon.feather')

In [ ]:
tripdistance = pd.read_feather('df_with_trip_distances.feather')

In [ ]:
tripdistance = tripdistance[['start_time','end_time','trip_distances']]

In [ ]:
df2 = tripdistance.merge(df, how = 'left' , left_on = ['start_time','end_time'], right_on= ['start_time','end_time'])

In [ ]:
# seperating data set for pre and post covid analysis 

precovidf =df2.loc[(df2['year']== 2018) |( df2['year'] == 2019)]
postcovidf =df2.loc[(df2['year']== 2020) |( df2['year'] == 2021)]
precovidf['group'] = list(zip(precovidf.start_coords, precovidf.end_coords))
precovidf['final_group']= precovidf.group.apply(lambda i: str(i[1][1]) + str(i[0][0]) + str(i[0][1])+ str(i[1][0]))
precovidf.shape
postcovidf['group'] = list(zip(postcovidf.start_coords, postcovidf.end_coords))
postcovidf['final_group']= postcovidf.group.apply(lambda i: str(i[1][1]) + str(i[0][0]) + str(i[0][1])+ str(i[1][0]))
print(precovidf.shape)
print(postcovidf.shape)

In [ ]:
# number of rides between each distinct start and end coordinates
precovrides = precovidf.groupby('final_group').count()[['group']] 
postcovrides = postcovidf.groupby('final_group').count()[['group']] 

len(precovrides)

precovrides.rename(columns= {'group':'num_rides'},inplace = True)
postcovrides.rename(columns= {'group':'num_rides'},inplace = True)

precovrides.reset_index(inplace=True)
postcovrides.reset_index(inplace=True)

In [ ]:
# merging pre and post covid data to get popularaity or number unique rides from one station to another 

pre_df= precovrides.merge(precovidf, how = 'left', left_on = 'final_group', right_on= 'final_group')
post_df= postcovrides.merge(postcovidf, how = 'left', left_on = 'final_group', right_on= 'final_group')

In [ ]:
print(pre_df.shape)
print(post_df.shape)
print(post_df.columns)
print(post_df.isna().sum())


In [ ]:
# data set with only unique station.
prestations =pre_df.drop_duplicates(['start_name'])
poststations =post_df.drop_duplicates(['start_name'])
print(prestations.shape)
print(poststations.shape)

In [ ]:
# finding number of rides per start station  or the popularity for color mapping 

ridesdf = pre_df.groupby('start_name').count()[['end_coords']]

ridesdf.rename(columns ={'end_coords': 'Popularity'}, inplace = True)
ridesdf.reset_index(inplace = True)

postridesdf = post_df.groupby('start_name').count()[['end_coords']]

postridesdf.rename(columns ={'end_coords': 'Popularity'}, inplace = True)
postridesdf.reset_index(inplace = True)

In [ ]:
# merging popularity back onto original pre and post covid data frames 
final_pre_df= ridesdf.merge(pre_df, how = 'left', left_on = 'start_name', right_on= 'start_name')
final_post_df= postridesdf.merge(post_df, how = 'left', left_on = 'start_name', right_on= 'start_name')
print(final_post_df.shape)
print(final_post_df.columns)
print(final_post_df.isna().sum())
print(final_pre_df.shape)

In [ ]:
df2.shape
ridesdf.shape
pd.merge?

In [ ]:
pd.concat([df2, ], axis=0)

In [ ]:
# saving final pre and post data to feather files 
# final_pre_df.to_feather('pre_df.feather')
# final_post_df.to_feather('post_df.feather')

In [ ]:
print(final_pre_df.isna().sum())

In [ ]:
# lopping thru popularity to assign color values based on ranges - pre covid 
ridesdf['color_map'] = ''
for i in range(len(ridesdf.index)):
    if ridesdf.Popularity.iloc[i]  >= 50000: ridesdf.loc[i,'color_map']= 'red'
    elif (ridesdf.Popularity.iloc[i]  < 50000 )& (ridesdf.Popularity.iloc[i]> 20000 ): ridesdf.loc[i,'color_map']='orange'
    elif( ridesdf.Popularity.iloc[i]  < 20000 ) & (ridesdf.Popularity.iloc[i] >5000) : ridesdf.loc[i,'color_map']='yellow'
    elif (ridesdf.Popularity.iloc[i]  <5000) & (ridesdf.Popularity.iloc[i] >1000):ridesdf.loc[i,'color_map']= 'green'
    elif (ridesdf.Popularity.iloc[i] <1000) & (ridesdf.Popularity.iloc[i] > 100):ridesdf.loc[i,'color_map']= 'blue'
    else :ridesdf.loc[i,'color_map']= 'purple'
    

In [ ]:
# post covid 
postridesdf['color_map'] = ''
for i in range(len(postridesdf.index)):
    if postridesdf.Popularity.iloc[i]  >= 50000: postridesdf.loc[i,'color_map']= 'red'
    elif (postridesdf.Popularity.iloc[i]  < 50000 )& (postridesdf.Popularity.iloc[i]> 20000 ): postridesdf.loc[i,'color_map']='orange'
    elif( postridesdf.Popularity.iloc[i]  < 20000 ) & (postridesdf.Popularity.iloc[i] >5000) : postridesdf.loc[i,'color_map']='yellow'
    elif (postridesdf.Popularity.iloc[i]  <5000) & (postridesdf.Popularity.iloc[i] >1000):postridesdf.loc[i,'color_map']= 'green'
    elif (postridesdf.Popularity.iloc[i] <1000) & (postridesdf.Popularity.iloc[i] > 100):postridesdf.loc[i,'color_map']= 'blue'
    else :postridesdf.loc[i,'color_map']= 'purple'

In [ ]:
# merging back onto unique indivudal stations to get coordinate pairs 

pre_popdf = prestations.merge(ridesdf, how = 'right', left_on = 'start_name', right_on = 'start_name')
post_popdf = poststations.merge(postridesdf, how = 'right', left_on = 'start_name', right_on = 'start_name')

In [ ]:
print(pre_popdf.shape)
print(post_popdf.shape)
print(ridesdf.shape)
print(postridesdf.shape)

In [ ]:
# pre and post data with only unique paths 
pre_df1 = pre_df.drop_duplicates(subset = ['final_group'])
post_df1 = post_df.drop_duplicates(subset = ['final_group'])


In [ ]:
print(pre_df1.shape)
print(post_df1.shape)

In [ ]:
# ensuring all duplicates are dropped 

pre_df1.drop_duplicates(subset = ['start_name','end_name','num_rides'],inplace =True)
post_df1.drop_duplicates(subset = ['start_name','end_name','num_rides'],inplace =True)

In [ ]:
print(pre_df1.shape)

In [ ]:
pre_df1.reset_index(inplace=True)
post_df1.reset_index(inplace=True)

In [ ]:
pre_df1.num_rides.sort_values(ascending = False)


In [ ]:
# unique station coordinates pre and post covid 
pree =pre_df1.drop_duplicates(['start_name'])
posts =post_df1.drop_duplicates(['start_name'])
print(pree.shape)
print(posts.shape)

In [ ]:
## geting boro locations for low density dockstations 

In [ ]:
import geopandas
from shapely.geometry import Polygon, Point
import geojson
north = [(40.872944, -73.927418),(40.895429, -73.917247),(40.884413, -73.888250),(40.853074, -73.815674)
              ,(40.813691, -73.816279),(40.798995, -73.908538),(40.810716, -73.932721),(40.837119, -73.931637)]

north = Polygon(north)

astoria =[(40.778900, -73.924769),(40.767786, -73.911384),(40.753993, -73.912774),
          (40.762848, -73.860803),(40.790865, -73.907390)]
astoria = Polygon(astoria)

redhook =[(40.667686, -74.002683),(40.640777, -73.955835),(40.603101, -74.016806),(40.628058, -74.048852)]
redhook = Polygon(redhook)
flushing =[(40.725841, -73.880005),(40.702408, -73.928458),(40.678606, -73.922120),(40.704628, -73.856625)]
flushing = Polygon(flushing) 


print(redhook)
print(flushing)
print(north)
print(astoria)


In [ ]:
post.reset_index(inplace =True)

In [ ]:
pre.reset_index(inplace=True)

In [ ]:
pre_exp=pre.loc[(pre['color_map'] =='blue')|(pre['color_map']== 'purple')|(pre['color_map'] == 'green')][['start_name', 'end_name', 'end_coords',
                                                                                                          'expansion','final_group', 'start_time','end_time','end_lng','end_lat']]
post_exp=post.loc[(post['color_map'] =='blue')|(post['color_map']== 'purple')|(post['color_map'] == 'green')][['start_name', 'end_name', 'end_coords','final_group',
                                                                                                               'start_time','end_time','end_lat','end_lng']]

In [ ]:
pre_exp.drop_duplicates(subset = ['start_name'],inplace=True)
post_exp.drop_duplicates(subset=['start_name'],inplace=True)
print(post.columns)
print(pre_exp.columns)
print(pre_exp.shape)
print(post_exp.shape)

In [ ]:

exp_pre1.end_lng =exp_pre1.end_lng.apply(lambda x:float(x))
exp_pre1.end_lat=exp_pre1.end_lat.apply(lambda x:float(x))

exp_post1.end_lng=exp_post1.end_lng.apply(lambda x:float(x))
exp_post1.end_lat=exp_post1.end_lat.apply(lambda x:float(x))


In [ ]:
exp_pre1.reset_index(inplace=True)
exp_post1.reset_index(inplace=True)

In [ ]:
# post exapansion area keys 

from shapely.geometry import Polygon, Point

exp_pre1['expansion'] = 0 

for z,i in enumerate(list(zip(exp_pre1.end_lat,exp_pre1.end_lng))):
    if Point(i).within(astoria):
        print(True)
        exp_pre1.loc[z,'expansion'] = 2
    
    elif Point(i).within(north):
         exp_pre1.loc[z,'expansion'] = 1
    elif Point(i).within(redhook):
        exp_pre1.loc[z,'expansion'] = 4
    elif Point(i).within(flushing): 
        exp_pre1.loc[z,'expansion'] = 3 

    print(z)

In [ ]:
# pre expansion area keys 

from shapely.geometry import Polygon, Point
exp_post1['expansion'] = 0

for z,i in enumerate(list(zip(exp_post1.end_lat,exp_post1.end_lng))):
    if Point(i).within(astoria):
         exp_post1.loc[z,'expansion'] = 2
    elif Point(i).within(north):
         exp_post1.loc[z,'expansion'] = 1
    elif Point(i).within(redhook):
        exp_post1.loc[z,'expansion'] = 4
    elif Point(i).within(flushing): 
        exp_post1.loc[z,'expansion'] = 3 
  
    print(z)
            
       
      

In [ ]:
# pre covid 

In [ ]:
exp_stsP=['St Johns Pl & Washington Ave',


'Graham Ave & Withers St', 'St Johns Pl & Washington Ave',
       'Lewis Ave & Madison St', 'N 11 St & Kent Ave',
       'Cambridge Pl & Gates Ave',


'Graham Ave & Withers St', '3 Ave & E 71 St', 'India St & West St',
       '31 St & Newtown Ave', 'N 11 St & Kent Ave']

In [ ]:
# post covid 

In [ ]:
exp2019 = pre_exp.loc[(pre_exp.start_name_x.isin(exp_stsP)) &(pre_exp.expansion_y !=0)]

In [ ]:
exp2019.head()

In [ ]:
exp2020 = post_exp.loc[post_exp.start_name_x.isin(exp_sts) & (post_exp.expansion_y !=0)]

In [ ]:
exp2020.sort_values(by='num_rides',ascending = True).reset_index(inplace= True)

In [ ]:
exp2020.loc[exp2020['expansion_y']==4][['start_name_x','num_rides','end_name']].sort_values(by='num_rides',ascending = False)

In [ ]:
len(exp2020.drop_duplicates(subset=['final_group_x']))

In [ ]:
#4
exp_sts= ['1 Pl & Clinton St' ,           
'1 St & 6 Ave'        ,           
'10 St & 2 Ave'        ,       
'11 Ave & Prospect Ave' ,        
'10 Hudson Yards',


'1 Pl & Clinton St',        
'1 St & 6 Ave'      ,       
'11 St & 43 Ave'      ,     
'10 Hudson Yards'    ,      
'2 Ave & E 122 St'     ,    
'10 St & 2 Ave'         ,   
'11 Ave & Prospect Ave'  ,  
'19 St & 24 Ave '  ,


'1 Pl & Clinton St' ,            
'1 St & 6 Ave'       ,           
'10 Hudson Yards'     ,          
'11 St & 43 Ave'       ,         
'10 St & 5 Ave'         ,      
'11 Ave & Prospect Ave'  ,        
'10 St & 2 Ave'           ,       
'11 St & 35 Ave'           ,     
'2 Ave & E 125 St'          ,     
'2 Ave & E 122 St'           ,    
'17 St & 5 Ave'               ,  
'10 Ave & W 207 St',


'1 Pl & Clinton St ',       
'2 Ave & E 122 St'   ,      
'11 St & 43 Ave'      ,    
'10 Hudson Yards'      ,    
'19 St & 24 Ave'        ,   
'10 St & 2 Ave'          , 
'10 Ave & W 202 St'] 

In [ ]:
pre_exp.loc[]

In [ ]:
pre_exp = pre_exp.merge(pre, how = "right", left_on = 'end_name', right_on= 'end_name')
post_exp = post_exp.merge(post, how = "right", left_on = 'end_name', right_on= 'end_name')
post_exp.expansion_y.value_counts()
pre_exp.expansion_y.value_counts()



In [ ]:
print(pre_exp.loc[pre_exp['expansion_y']==4].start_name_x.value_counts().index)
print(pre_exp.loc[pre_exp['expansion_y']==3].start_name_x.value_counts().index)
print(pre_exp.loc[pre_exp['expansion_y']==2].start_name_x.value_counts().index)
print(pre_exp.loc[pre_exp['expansion_y']==1].start_name_x.value_counts().index)

In [ ]:
print(post_exp.loc[post_exp['expansion_y']==4].start_name_x.value_counts())
print(post_exp.loc[post_exp['expansion_y']==3].start_name_x.value_counts())
print(post_exp.loc[post_exp['expansion_y']==2].start_name_x.value_counts())
print(post_exp.loc[post_exp['expansion_y']==1].start_name_x.value_counts())

In [ ]:
pre_exp = pre[['expansion', 'start_lng', 'end_lng', 'start_lat', 'end_lat', 
               'start_coords', 'end_coords', 'num_rides', 'Popularity', 'name_end',
               'start_name', 'color_map', 'final_group']]

# post_exp = post[['expansion', 'start_lng', 'end_lng', 'start_lat', 'end_lat', 
#                'start_coords', 'end_coords', 'num_rides', 'Popularity', 'name_end',
#                'start_name', 'color_map', 'final_group']]

# post_exp.to_feather('post_exp.feather')

# pre_exp.to_feather('pre_exp.feather')

In [ ]:
nsdf = pre.loc[pre['Popularity']< 300][['start_name','start_lat', 'start_lng','end_lng','end_lat', 'Popularity', 'num_rides']]
nsdf= nsdf.drop_duplicates(subset=['start_name'])
nsdf.reset_index(inplace=True)

In [ ]:
pre.loc[(pre['color_map']=='blue' )|(pre['color_map']=='purple' )| pre['color_map']=='green']

In [ ]:
# Searching for important expansion areas by borough 

In [7]:
zips = pd.read_csv("zip_borough.csv")

In [8]:
zips['end_boro'] = zips.borough
zipsend = zips[['end_boro', 'zip']]

In [9]:
zips['start_boro'] = zips.borough 

In [10]:
zipstart = zips[['start_boro', 'zip']]

In [11]:
# dropping inconclusive zip codes 
postzip = post[post.zipcode_start!='H3A']
prezip = pre[pre.zipcode_start!='H3A']

In [242]:
postzip.zipcode_start.value_counts()
prezip.zipcode_start.value_counts()

10014     8065951
10037      105805
10019       97311
10010       92445
10011       90179
10003       76422
10005       73267
10016       66613
10009       63856
10022       51279
10018       49321
10128       47624
10034       47198
11201       44974
10024       40069
10119       39611
10001       38521
10025       37471
10036       36964
10002       32823
10038       30870
10012       24811
10017       22409
10013       20869
11217       17810
11101       16884
10021       15846
10045       13707
10279       13500
10105       13005
11213       12511
10007       10817
10004       10614
11221        9746
11222        9146
11207        8195
11249        8122
11238        7548
11225        6971
10028        6468
11231        5733
112321       4878
10115        4575
10029        3837
11369        3271
10023        3219
11205        2580
10035        2568
11215        2067
10110        1533
11251        1213
11211         769
11106         708
10075         694
11102         564
10275     

In [12]:
# switching zip columns to int for the merge for boroughts 
import re
postzip.zipcode_start=postzip.zipcode_start.apply(lambda x : int(re.split(('-|:'),x)[0]))
prezip.zipcode_start = prezip.zipcode_start.apply(lambda x :  int(re.split(('-|:'),x)[0]))

<ipython-input-12-8035e3a1c651>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  prezip.zipcode_start = prezip.zipcode_start.apply(lambda x :  int(re.split(('-|:'),x)[0]))


In [13]:
postzip.zipcode_end=postzip.zipcode_end.apply(lambda x : int(re.split(('-|:'),x)[0]))
prezip.zipcode_end = prezip.zipcode_end.apply(lambda x :  int(re.split(('-|:'),x)[0]))

<ipython-input-13-60cca43b043e>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  prezip.zipcode_end = prezip.zipcode_end.apply(lambda x :  int(re.split(('-|:'),x)[0]))


,start_boro,zip
135,Queens,11101


In [14]:
postzip = postzip.merge(zipsend, how = 'left' , left_on = 'zipcode_end', right_on = 'zip')

prezip = prezip.merge(zipsend, how = 'left' , left_on = 'zipcode_end', right_on = 'zip')

In [15]:
postzip = postzip.merge(zipstart, how = 'left' , left_on = 'zipcode_start', right_on = 'zip')

prezip = prezip.merge(zipstart, how = 'left' , left_on = 'zipcode_start', right_on = 'zip')

In [18]:
postzip.loc[postzip.zip_x == 11101]

,start_name,Popularity,color_map,final_group,num_rides,start_time,end_time,trip_distances,duration,start_lat,...,coords_start,zipcode_start,name_end,coords_end,zipcode_end,group,end_boro,zip_x,start_boro,zip_y
9471,1 Ave & E 16 St,35204,orange,-73.94609340.732219-73.98165640.748,1,2021-06-28 14:32:01.000,2021-06-28 14:52:13.000,2.484771,1212,40.732219,...,"[40.732219, -73.981656]",10014,44 Dr & 21 St,"[40.748, -73.946093]",11101,"[[40.732219, -73.981656], [40.748, -73.946093]]",Queens,11101.0,Manhattan,10014.0
44467,1 Ave & E 18 St,29842,orange,-73.93725940.733812-73.98054440.741283,3,2020-11-19 18:44:36.112,2020-11-19 19:18:03.650,3.005225,2007,40.733812,...,"[40.733812, -73.980544]",10014,48 Ave & 30 Pl,"[40.741283, -73.937259]",11101,"[[40.733812, -73.980544], [40.741283, -73.9372...",Queens,11101.0,Manhattan,10014.0
44468,1 Ave & E 18 St,29842,orange,-73.93725940.733812-73.98054440.741283,3,2020-11-12 18:38:20.053,2020-11-12 19:05:14.426,3.005225,1614,40.733812,...,"[40.733812, -73.980544]",10014,48 Ave & 30 Pl,"[40.741283, -73.937259]",11101,"[[40.733812, -73.980544], [40.741283, -73.9372...",Queens,11101.0,Manhattan,10014.0
44469,1 Ave & E 18 St,29842,orange,-73.93725940.733812-73.98054440.741283,3,2020-10-15 18:11:56.434,2020-10-15 18:46:16.768,3.005225,2060,40.733812,...,"[40.733812, -73.980544]",10014,48 Ave & 30 Pl,"[40.741283, -73.937259]",11101,"[[40.733812, -73.980544], [40.741283, -73.9372...",Queens,11101.0,Manhattan,10014.0
74439,1 Ave & E 30 St,21614,orange,-73.93725940.741443-73.9753640.741283,1,2021-02-20 13:55:52.000,2021-02-20 14:43:16.000,2.642301,2844,40.741443,...,"[40.741443, -73.97536]",10016,48 Ave & 30 Pl,"[40.741283, -73.937259]",11101,"[[40.741443, -73.97536], [40.741283, -73.937259]]",Queens,11101.0,Manhattan,10016.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11553366,Wythe Ave & Metropolitan Ave,22324,orange,-73.94609340.716887-73.96319840.748,8,2021-09-15 18:16:43.000,2021-09-15 18:31:54.000,1.327846,911,40.716887,...,"[40.716887, -73.963198]",10014,44 Dr & 21 St,"[40.748, -73.946093]",11101,"[[40.716887, -73.963198], [40.748, -73.946093]]",Queens,11101.0,Manhattan,10014.0
11553367,Wythe Ave & Metropolitan Ave,22324,orange,-73.94609340.716887-73.96319840.748,8,2021-09-05 10:35:57.000,2021-09-05 10:54:47.000,1.327846,1130,40.716887,...,"[40.716887, -73.963198]",10014,44 Dr & 21 St,"[40.748, -73.946093]",11101,"[[40.716887, -73.963198], [40.748, -73.946093]]",Queens,11101.0,Manhattan,10014.0
11553368,Wythe Ave & Metropolitan Ave,22324,orange,-73.94609340.716887-73.96319840.748,8,2020-11-26 12:01:03.765,2020-11-26 12:14:57.077,1.327846,833,40.716887,...,"[40.716887, -73.963198]",10014,44 Dr & 21 St,"[40.748, -73.946093]",11101,"[[40.716887, -73.963198], [40.748, -73.946093]]",Queens,11101.0,Manhattan,10014.0
11553369,Wythe Ave & Metropolitan Ave,22324,orange,-73.94609340.716887-73.96319840.748,8,2020-10-11 16:09:08.122,2020-10-11 16:27:43.203,1.327846,1115,40.716887,...,"[40.716887, -73.963198]",10014,44 Dr & 21 St,"[40.748, -73.946093]",11101,"[[40.716887, -73.963198], [40.748, -73.946093]]",Queens,11101.0,Manhattan,10014.0


In [129]:
# brooklyn pre start
px.bar(prezip.loc[(prezip.start_boro == 'Brooklyn')][['start_name','Popularity']].value_counts().to_frame().reset_index().sort_values(by='Popularity', ascending=False).head(), x= 'start_name',y='Popularity', labels ={'start_name':'station start name'})

In [130]:
# brookly post start
px.bar(postzip.loc[(postzip.start_boro == 'Brooklyn')][['start_name','Popularity']].value_counts().to_frame().reset_index().sort_values(by='Popularity', ascending=False).head(), x= 'start_name',y='Popularity', labels ={'start_name':'station start name'})

In [131]:
# bronx pre start - no stations
px.bar(prezip.loc[(prezip.start_boro == 'Bronx')][['start_name','Popularity']].value_counts().to_frame().reset_index().sort_values(by='Popularity', ascending=False).head(), x= 'start_name',y='Popularity', labels ={'start_name':'station start name'})

In [132]:
# bronx post start
px.bar(postzip.loc[(postzip.start_boro == 'Bronx')][['start_name','Popularity']].value_counts().to_frame().reset_index().sort_values(by='Popularity', ascending=False).head(), x= 'start_name',y='Popularity', labels ={'start_name':'station start name'})

In [128]:
# pre start manhattan
px.bar(prezip.loc[(prezip.start_boro == 'Manhattan')][['start_name','Popularity']].value_counts().to_frame().reset_index().sort_values(by='Popularity', ascending=False).head(), x= 'start_name',y='Popularity', labels ={'start_name':'station start name'})

In [136]:
# manhaatan psot start
px.bar(postzip.loc[(postzip.start_boro == 'Manhattan')][['start_name','Popularity']].value_counts().to_frame().reset_index().sort_values(by='Popularity', ascending=False).head(), x= 'start_name',y='Popularity', labels ={'start_name':'station start name'})

In [137]:
# queens start pre 
px.bar(prezip.loc[(prezip.start_boro == 'Queens')][['start_name','Popularity']].value_counts().to_frame().reset_index().sort_values(by='Popularity', ascending=False).head(), x= 'start_name',y='Popularity', labels ={'start_name':'station start name'})

In [138]:
# queens start post 
px.bar(postzip.loc[(postzip.start_boro == 'Queens')][['start_name','Popularity']].value_counts().to_frame().reset_index().sort_values(by='Popularity', ascending=False).head(), x= 'start_name',y='Popularity', labels ={'start_name':'station start name'})

In [20]:
postzip.loc[(postzip.start_boro == 'Queens')][['start_name','end_boro','start_boro']].value_counts()

start_name                        end_boro   start_boro
44 Dr & 21 St                     Manhattan  Queens        4275
Queens Plaza North & Crescent St  Manhattan  Queens        3588
44 Dr & Jackson Ave               Manhattan  Queens        1542
Center Blvd & 51 Ave              Manhattan  Queens        1425
31 St & Newtown Ave               Manhattan  Queens        1382
                                                           ... 
49 Ave & 21 St                    Queens     Queens           1
Flushing Ave & Woodward Ave       Brooklyn   Queens           1
Crescent St & Ditmars Blvd        Queens     Queens           1
8 St & Astoria Blvd               Queens     Queens           1
48 Ave & 5 St                     Queens     Queens           1
Length: 140, dtype: int64

In [24]:
prezip.loc[(prezip.end_boro == 'Queens')][['start_name','end_name','end_boro','start_boro','zip_x']].value_counts()

start_name                end_name                  end_boro  start_boro  zip_x  
46 Ave & 5 St             44 Dr & 21 St             Queens    Manhattan   11101.0    1015
28 St & 41 Ave            28 St & 38 Ave            Queens    Manhattan   11101.0     276
31 St & Broadway          31 St & 35 Ave            Queens    Manhattan   11101.0     266
Center Blvd & 48 Ave      44 Dr & 21 St             Queens    Manhattan   11101.0     238
27 St & Hunter St         24 St & 41 Ave            Queens    Manhattan   11101.0     228
                                                                                     ... 
E 48 St & 3 Ave           11 St & 35 Ave            Queens    Manhattan   11106.0       1
E 47 St & Park Ave        48 Ave & 30 Pl            Queens    Manhattan   11101.0       1
                          31 St & 35 Ave            Queens    Manhattan   11101.0       1
E 47 St & 2 Ave           Center Blvd & Borden Ave  Queens    Manhattan   11101.0       1
old - E 13 St & Av

In [255]:
prezip.loc[(prezip.start_boro == 'Queens')][['end_name', 'end_boro','start_boro']].value_counts()

end_name                          end_boro   start_boro
46 Ave & 5 St                     Manhattan  Queens        1058
Queens Plaza North & Crescent St  Manhattan  Queens         681
28 St & 41 Ave                    Manhattan  Queens         664
31 St & Broadway                  Manhattan  Queens         646
44 Dr & Jackson Ave               Manhattan  Queens         559
                                                           ... 
Lexington Ave & E 111 St          Manhattan  Queens           1
St Johns Pl & Washington Ave      Manhattan  Queens           1
Lexington Ave & E 127 St          Manhattan  Queens           1
Lexington Ave & E 24 St           Manhattan  Queens           1
1 Ave & E 110 St                  Manhattan  Queens           1
Length: 599, dtype: int64

In [258]:
# bronx most popular stations 

# Pre covid 
print(prezip.loc[(prezip.start_boro == 'Bronx')][['start_name','end_boro','start_boro']].value_counts())
print(prezip.loc[(prezip.end_boro == 'Bronx')][['start_name','end_boro','start_boro']].value_counts())
print(prezip.loc[(prezip.start_boro == 'Brooklyn')][['start_name','end_boro','start_boro']].value_counts())
print(prezip.loc[(prezip.end_boro == 'Brooklyn')][['end_name','end_boro','start_boro']].value_counts())

# post covid 
print(postzip.loc[(postzip.start_boro == 'Brooklyn')][['start_name','end_boro','start_boro']].value_counts())
print(postzip.loc[(postzip.end_boro == 'Brooklyn')][['end_name','end_boro','start_boro']].value_counts())
print(postzip.loc[(postzip.end_boro == 'Brooklyn')][['start_name','end_boro','start_boro']].value_counts())


Series([], dtype: int64)
Series([], dtype: int64)
start_name                       end_boro   start_boro
Dean St & 4 Ave                  Manhattan  Brooklyn      11083
Cadman Plaza West & Montague St  Manhattan  Brooklyn       9250
Lafayette Ave & Fort Greene Pl   Manhattan  Brooklyn       8927
State St & Smith St              Manhattan  Brooklyn       8084
Congress St & Clinton St         Manhattan  Brooklyn       7110
                                                          ...  
Wyckoff St & 3 Ave               Queens     Brooklyn          1
Lorimer St & Calyer St           Brooklyn   Brooklyn          1
Lafayette Ave & Fort Greene Pl   Queens     Brooklyn          1
Henry St & W 9 St (OLD)          Brooklyn   Brooklyn          1
Henry St & Poplar St             Queens     Brooklyn          1
Length: 102, dtype: int64
end_name                                      
Dean St & 4 Ave                                   12554
Lafayette Ave & Fort Greene Pl                     9966
Cadman

In [196]:
postzip.loc[(postzip.end_boro == 'Queens')|(postzip.end_boro == 'Bronx')][['start_name]]'.value_counts()

,start_name,Popularity,color_map,final_group,num_rides,start_time,end_time,trip_distances,duration,start_lat,...,day_type,name_start,coords_start,zipcode_start,name_end,coords_end,zipcode_end,group,zip,borough
164,1 Ave & E 110 St,9085,yellow,-73.9176940.792327-73.938340.828847,2,2020-12-05 18:00:44.841,2020-12-05 18:27:19.784,1.592172,1594,40.792327,...,Weekend,1 Ave & E 110 St,"[40.792327, -73.9383]",10014,Grant Ave & E 164 St,"[40.828847, -73.91769]",10456,"[[40.792327, -73.9383], [40.828847, -73.91769]]",10456.0,Bronx
165,1 Ave & E 110 St,9085,yellow,-73.9176940.792327-73.938340.828847,2,2020-09-30 15:22:41.093,2020-09-30 15:50:59.992,1.592172,1698,40.792327,...,Weekday,1 Ave & E 110 St,"[40.792327, -73.9383]",10014,Grant Ave & E 164 St,"[40.828847, -73.91769]",10456,"[[40.792327, -73.9383], [40.828847, -73.91769]]",10456.0,Bronx
265,1 Ave & E 110 St,9085,yellow,-73.9227240.792327-73.938340.827115,3,2021-04-05 15:44:33.000,2021-04-05 16:01:38.000,1.270364,1025,40.792327,...,Weekday,1 Ave & E 110 St,"[40.792327, -73.9383]",10014,Grand Concourse & E 161 St,"[40.827115, -73.92272]",10451,"[[40.792327, -73.9383], [40.827115, -73.92272]]",10451.0,Bronx
266,1 Ave & E 110 St,9085,yellow,-73.9227240.792327-73.938340.827115,3,2020-09-12 18:36:41.515,2020-09-12 18:57:09.561,1.270364,1228,40.792327,...,Weekend,1 Ave & E 110 St,"[40.792327, -73.9383]",10014,Grand Concourse & E 161 St,"[40.827115, -73.92272]",10451,"[[40.792327, -73.9383], [40.827115, -73.92272]]",10451.0,Bronx
267,1 Ave & E 110 St,9085,yellow,-73.9227240.792327-73.938340.827115,3,2020-10-14 23:38:49.310,2020-10-15 00:00:28.063,1.270364,1298,40.792327,...,Weekday,1 Ave & E 110 St,"[40.792327, -73.9383]",10014,Grand Concourse & E 161 St,"[40.827115, -73.92272]",10451,"[[40.792327, -73.9383], [40.827115, -73.92272]]",10451.0,Bronx
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11553366,Wythe Ave & Metropolitan Ave,22324,orange,-73.94609340.716887-73.96319840.748,8,2021-09-15 18:16:43.000,2021-09-15 18:31:54.000,1.327846,911,40.716887,...,Weekday,Wythe Ave & Metropolitan Ave,"[40.716887, -73.963198]",10014,44 Dr & 21 St,"[40.748, -73.946093]",11101,"[[40.716887, -73.963198], [40.748, -73.946093]]",11101.0,Queens
11553367,Wythe Ave & Metropolitan Ave,22324,orange,-73.94609340.716887-73.96319840.748,8,2021-09-05 10:35:57.000,2021-09-05 10:54:47.000,1.327846,1130,40.716887,...,Weekend,Wythe Ave & Metropolitan Ave,"[40.716887, -73.963198]",10014,44 Dr & 21 St,"[40.748, -73.946093]",11101,"[[40.716887, -73.963198], [40.748, -73.946093]]",11101.0,Queens
11553368,Wythe Ave & Metropolitan Ave,22324,orange,-73.94609340.716887-73.96319840.748,8,2020-11-26 12:01:03.765,2020-11-26 12:14:57.077,1.327846,833,40.716887,...,Weekday,Wythe Ave & Metropolitan Ave,"[40.716887, -73.963198]",10014,44 Dr & 21 St,"[40.748, -73.946093]",11101,"[[40.716887, -73.963198], [40.748, -73.946093]]",11101.0,Queens
11553369,Wythe Ave & Metropolitan Ave,22324,orange,-73.94609340.716887-73.96319840.748,8,2020-10-11 16:09:08.122,2020-10-11 16:27:43.203,1.327846,1115,40.716887,...,Weekend,Wythe Ave & Metropolitan Ave,"[40.716887, -73.963198]",10014,44 Dr & 21 St,"[40.748, -73.946093]",11101,"[[40.716887, -73.963198], [40.748, -73.946093]]",11101.0,Queens


In [49]:
post2020 = post[post['year']==2020]
post2021 = post[post['year']==2021]
pre2018 = pre[pre['year']==2018]
pre2019 = pre[pre['year']==2019]

In [53]:
pre2018[['bus_dist_start', 'high_traffic_start', 'comm_dist_start',
           'park_dist_start', 'bus_dist_end', 'high_traffic_end', 'subway_dist_end', 'park_dist_end']].describe()

,bus_dist_start,high_traffic_start,comm_dist_start,park_dist_start,bus_dist_end,high_traffic_end,subway_dist_end,park_dist_end
count,4.385348e+06,4.385348e+06,4.385348e+06,4.385348e+06,4.385348e+06,4.385348e+06,4.385348e+06,4.385348e+06
mean,1.036199e-01,1.945232e-01,2.990476e-03,1.516516e-03,1.041545e-01,1.957316e-01,1.934320e-01,1.525139e-03
std,7.310369e-01,3.958333e-01,1.105125e-02,1.063291e-02,7.310906e-01,3.967628e-01,7.417808e-01,1.063337e-02
min,2.780319e-03,0.000000e+00,0.000000e+00,0.000000e+00,2.780319e-03,0.000000e+00,2.497367e-03,0.000000e+00
25%,4.430982e-02,0.000000e+00,2.318720e-04,2.361488e-04,4.440190e-02,0.000000e+00,8.949462e-02,2.386661e-04
50%,8.464273e-02,0.000000e+00,2.222142e-03,1.245491e-03,8.515416e-02,0.000000e+00,1.594276e-01,1.259769e-03
75%,1.427776e-01,0.000000e+00,4.423978e-03,2.345048e-03,1.427776e-01,0.000000e+00,2.607563e-01,2.345048e-03
max,3.175512e+02,1.000000e+00,4.633000e+00,4.604722e+00,3.175512e+02,1.000000e+00,3.180887e+02,4.604722e+00


In [54]:
exp_pre1[['bus_dist_start', 'high_traffic_start', 'comm_dist_start',
           'park_dist_start', 'bus_dist_end', 'high_traffic_end', 'subway_dist_end', 'park_dist_end']].describe()

,bus_dist_start,high_traffic_start,comm_dist_start,park_dist_start,bus_dist_end,high_traffic_end,subway_dist_end,park_dist_end
count,79.000000,79.000000,79.000000,79.000000,79.000000,79.000000,79.000000,79.000000
mean,0.139966,0.126582,0.004203,0.001980,0.127724,0.189873,0.200114,0.002079
std,0.137114,0.334629,0.002982,0.001066,0.068299,0.394707,0.145591,0.001466
min,0.010270,0.000000,0.000000,0.000205,0.024880,0.000000,0.026081,0.000039
25%,0.051525,0.000000,0.000952,0.000923,0.082426,0.000000,0.093822,0.000858
50%,0.106666,0.000000,0.006509,0.002554,0.131849,0.000000,0.158325,0.002311
75%,0.155808,0.000000,0.006804,0.002959,0.155426,0.000000,0.307628,0.002957
max,0.437451,1.000000,0.007258,0.003478,0.300485,1.000000,0.562472,0.004959


In [75]:
exp_post1.loc[exp_post1['expansion']==4][['start_name','num_rides']].value_counts()

start_name             num_rides
10 St & 5 Ave          1            39
11 Ave & Prospect Ave  1            18
10 St & 5 Ave          2            18
1 St & 6 Ave           1            17
10 St & 2 Ave          1            14
                                    ..
10 St & 5 Ave          35            1
11 Ave & Prospect Ave  3             1
                       10            1
                       13            1
17 St & 5 Ave          82            1
Length: 67, dtype: int64

In [61]:
print(df.high_traffic_end.value_counts())
print(df.high_traffic_start.value_counts())
print(17195913 /(17195913+ 3897066) )
print(17224370/(17224370+ 3868609) )

0.0    17195913
1.0     3897066
Name: high_traffic_end, dtype: int64
0.0    17224370
1.0     3868609
Name: high_traffic_start, dtype: int64
0.8152434513873076
0.8165925732918048


In [65]:
print(exp_pre1.high_traffic_end.value_counts())
print(exp_pre1.high_traffic_start.value_counts())
print(64/(64+15))
print(69/(69+10))
print('######## Post covid #############')
print(exp_post1.high_traffic_end.value_counts())
print(exp_post1.high_traffic_start.value_counts())

0.0    64
1.0    15
Name: high_traffic_end, dtype: int64
0.0    69
1.0    10
Name: high_traffic_start, dtype: int64
0.810126582278481
0.8734177215189873
######## Post covid #############
0.0    774
1.0     28
Name: high_traffic_end, dtype: int64
0.0    596
1.0    206
Name: high_traffic_start, dtype: int64


In [64]:
print((774) / (774+ 28))
print((596) / (596+ 206))

0.9650872817955112
0.743142144638404


In [ ]:
# getting important exapansion area stations by polygon 

In [3]:
pre = pd.read_feather('pre_df.feather')
post = pd.read_feather('post_df.feather')

In [4]:
df = pd.read_feather('the_final_df.feather')


In [5]:
exp_pre1 = pd.read_feather('exp_data_pre.feather')
exp_post1 = pd.read_feather('exp_data_post.feather')

In [6]:
distinct_pre = pre.drop_duplicates(subset=['start_name'])
distinct_post = post.drop_duplicates(subset=['start_name'])
distinct_pre.reset_index(inplace=True)
distinct_post.reset_index(inplace = True)

In [ ]:
# dropping duplicate rides 
exp_pre = pre.drop_duplicates(subset=['final_group'])
exp_post = post.drop_duplicates(subset =['final_group'])
print(exp_pre.shape)
print(exp_post.shape)

In [ ]:
# filtering data for start stations with highest value counts of end stations in expansion areas 
exp_post1 =exp_post.loc[(exp_post.start_name.isin(exp_sts))]

In [ ]:
exp_pre1 = exp_pre.loc[(exp_pre.start_name.isin(exp_stsP))]
print(exp_pre1.shape)
print(exp_post1.shape)
print(exp_post1.isna().sum())

In [ ]:
# filtering the data for only end stations in exansion zones 
exp_post1 =exp_post1[exp_post1['expansion']!=0]
exp_pre1 =exp_pre1[exp_pre1['expansion']!=0]



In [ ]:
# sorting stations with the maximum number of rides 
exp_pre1= exp_pre1.sort_values(by='num_rides', ascending = False)
exp_post1=exp_post1.sort_values(by='num_rides', ascending = False)


In [ ]:
# saving data for future use in maps 

# exp_post1.reset_index(inplace=True)
# exp_pre1.reset_index(inplace=True)
exp_post1.index
exp_pre1.set_index('group',inplace =True)
exp_pre1.reset_index(inplace=True)

In [ ]:
# exp_pre1.to_feather('exp_data_pre.feather')
# exp_post1.to_feather('exp_data_post.feather')

In [ ]:
from plotly.offline import init_notebook_mode, iplot
init_notebook_mode(connected=True)  

In [41]:
figpost = go.Figure()


figpost.add_trace(go.Scattermapbox(
    lon = distinct_post['start_lng'],
    lat = distinct_post['start_lat'],
    hoverinfo = 'text',
    text = distinct_post['start_name'],
    mode = 'markers',
    marker = dict(
        size = 4,
        color = distinct_post['color_map']
        )))

for i in range(len(exp_post1)):
    figpost.add_trace(
        go.Scattermapbox(
            
            lon = [exp_post1['start_lng'].iloc[i], exp_post1['end_lng'].iloc[i]],
            lat = [exp_post1['start_lat'].iloc[i], exp_post1['end_lat'].iloc[i]],
            mode = 'lines',
            hoverinfo = 'text',
            text = exp_post1['start_name'].iloc[i],
            line = dict(width =( exp_post1.num_rides.iloc[i]/50 ),color = 'red')

          
        )
    )
figpost.update_geos(center=dict(lat = 40.7831, lon = -73.9712 ),fitbounds = 'locations')
figpost.update_layout(mapbox_style="open-street-map", mapbox = dict(center=dict(lat = 40.7831, lon = -73.9712 ), zoom =10), showlegend=False)

figpost.show()

In [ ]:
# Pre covid analysis 

In [27]:
figpre = go.Figure()


figpre.add_trace(go.Scattermapbox(
    lon = distinct_pre['start_lng'],
    lat = distinct_pre['start_lat'],
    hoverinfo = 'text',
    text = distinct_pre['start_name'],
    mode = 'markers',
    marker = dict(
        size = 4,
        color = distinct_pre['color_map']
        )
    ))

for i in range(len(exp_pre1)):
    figpre.add_trace(
        go.Scattermapbox(
            
            lon = [exp_pre1['start_lng'].iloc[i], exp_pre1['end_lng'].iloc[i]],
            lat = [exp_pre1['start_lat'].iloc[i], exp_pre1['end_lat'].iloc[i]],
            mode = 'lines',
            hoverinfo = 'text',
            text = exp_pre1['start_name'].iloc[i],
            line = dict(width =( exp_pre1.num_rides.iloc[i]/50 ),color = 'red')
          
        )
    )
figpre.update_geos(center=dict(lat = 40.7831, lon = -73.9712 ),fitbounds = 'locations')
figpre.update_layout(mapbox_style="open-street-map", mapbox = dict(center=dict(lat = 40.7831, lon = -73.9712 ), zoom =10), showlegend=False)

figpre.show()

In [111]:
figpre = go.Figure()


figpre.add_trace(go.Scattermapbox(
    lon = distinct_pre['start_lng'],
    lat = distinct_pre['start_lat'],
    hoverinfo = 'text',
    text = distinct_pre['start_name'],
    mode = 'markers',
    marker = dict(
        size = 4,
        color = distinct_pre['color_map']
        )
    ))

for i in range(len(exp_pre1)):
    figpre.add_trace(
        go.Scattermapbox(
            
            lon = [exp_pre1['start_lng'].iloc[i], exp_pre1['end_lng'].iloc[i]],
            lat = [exp_pre1['start_lat'].iloc[i], exp_pre1['end_lat'].iloc[i]],
            mode = 'lines',
            hoverinfo = 'text',
            text = [exp_pre1['start_name'].iloc[i]],
            line = dict(width =( exp_pre1.num_rides.iloc[i]/50 ),color = 'red')
          
        )
    )
figpre.update_geos(center=dict(lat = 40.7831, lon = -73.9712 ),fitbounds = 'locations')
figpre.update_layout(mapbox_style="open-street-map", mapbox = dict(center=dict(lat = 40.7831, lon = -73.9712 ), zoom =10), showlegend=False)

figpre.show()

In [ ]:
# post covid analysis

In [ ]:
import seaborn as sns

In [ ]:
dfmodel = df[df['year']==2021].sample(100000)

In [ ]:
df.columns

In [ ]:
dfmodel.end_lng =dfmodel.end_lng.apply(lambda x:float(x))
dfmodel.end_lat=dfmodel.end_lat.apply(lambda x:float(x))

dfmodel.start_lng=dfmodel.start_lng.apply(lambda x:float(x))
dfmodel.start_lat=dfmodel.start_lat.apply(lambda x:float(x))

In [ ]:
# organizing data for models 

features1 = [ 'user_type', 'month',
'year', 'day_of_week', 'hour_of_day', 
'subway_dist_end','bus_dist_end','high_traffic_start','park_dist_end',
'comm_dist_end','end_lng','end_lat'] 

features =[ 'user_type', 'month',
'year', 'day_of_week', 'hour_of_day', 
'subway_dist_start','bus_dist_start','high_traffic_start','park_dist_start',
'comm_dist_start','start_lng','start_lat'] 

y = dfmodel['durationmins']

dummies = pd.get_dummies(dfmodel['day_type'], drop_first=True)

x = pd.concat([dfmodel[features], dummies], axis = 1)


In [ ]:
# LINEAR REGRESSION  - Cant predict duration based on start distances and time 

from sklearn.linear_model import LinearRegression


lm = LinearRegression()
lm.fit(X_train,y_train)
print(lm.score(X_train,y_train))
print(lm.score(X_test,y_test))


In [ ]:
## random forest - unable to predict 

import sklearn.model_selection as ms
from sklearn import ensemble
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(x,y, test_size=0.2, random_state=0)


randomForest = ensemble.RandomForestRegressor()

randomForest.set_params(random_state=0, max_depth = 15)

randomForest.fit(X_train, y_train)

print(randomForest.score(X_train,y_train))
print(randomForest.score(X_test,y_test))



In [ ]:
# Grid Search 
grid_para_tree = [
    {'max_depth': range(1, 31)}
]

tree_model.set_params(random_state=0)
grid_search_tree = ms.GridSearchCV(tree_model.fit(X_train, y_train),grid_para_tree, scoring='r2', cv=3, n_jobs=-1, return_train_score=True)
grid_search_tree.fit(X_train, y_train)

In [ ]:
# DASH APP

In [78]:
import dash
from dash import Dash, html, dcc

In [ ]:

pre1 = pre.drop_duplicates(subset=['start_name']).sort_values(by = "Popularity", ascending = False).head(20)
post1 = post.drop_duplicates(subset=['start_name']).sort_values(by = "Popularity", ascending = False).head(20)
post1.reset_index(inplace =True)
pre1.reset_index(inplace =True)

In [79]:
#edas  For app 
import seaborn as sns
import matplotlib.pyplot as plt
from plotly.offline import init_notebook_mode, iplot

In [237]:
px.bar(pre[['Popularity','start_name']].sort_values(by = 'Popularity', ascending = False).drop_duplicates(subset =['start_name']).head(10), x = 'start_name', y='Popularity',labels={'start_name':"Station Name", "Popularity":"popularity"})



In [239]:
px.bar(post[['Popularity','start_name']].sort_values(by = 'Popularity', ascending = False).drop_duplicates(subset =['start_name']).head(10), x = 'start_name', y='Popularity',labels={'start_name':"Station Name", "Popularity":"popularity"})

In [ ]:
p = sns.displot(df.loc[df['durationmins'] <60 ],x = 'durationmins', hue='year', kind = 'kde', height=8.27, aspect=11.7/8.27).set(title='Distribution of Ride Duration by Year')

p.set_ylabel("Density", fontsize = 20)
plt.show()

In [ ]:
# distribution of ride duration weekend vs weekway by year - pre covid 


plot1 = sns.FacetGrid(pre[['day_type','year','hour_of_day']], col = 'day_type', hue = 'year', height=8.27, aspect=11.7/8.27)
plot1.map(sns.histplot, 'hour_of_day', bins = 20).add_legend() #.set(title='Distribution of Ride Duration by Year')


plt.show()

In [ ]:
# distribution of ride horus weekend vs weekway by year - post covid 


plot2 = sns.FacetGrid(post[['day_type','year','hour_of_day']], col = 'day_type', hue = 'year', height=8.27, aspect=11.7/8.27)
plot2.map(sns.histplot, 'hour_of_day', bins = 20).add_legend().set(title='Distribution of Ride Duration by Year')


plt.show()

In [240]:
# line graph of amount of user type per year 

figRide = px.line(df.groupby(['year', 'user_type']).size().to_frame().reset_index(), x = 'year', y = [0] 
              ,color = 'user_type',title="Annual Rides By User Type")
figRide.update_xaxes(nticks=4,
        title_text = "year",
        title_font = {"size": 20},
        title_standoff = 25)
newnames = {'0':'Casual', '1': 'Member'}
figRide.for_each_trace(lambda t: t.update(name = newnames[t.name],
                                      legendgroup = newnames[t.name],
                                      hovertemplate = t.hovertemplate.replace(t.name, newnames[t.name])))
figRide.update_layout(legend_title = 'User Type')

figRide.update_yaxes(
        title_text = "Total Number of Rides",
        title_standoff = 25)

In [144]:
# number of rides by user type and year 

figMonth = px.line(df.groupby(['month','year']).size().to_frame().reset_index(), x = 'month', y = [0] 
              ,color = 'year',title="Month Rides By User Type and Year")
figMonth.update_xaxes(nticks=12,
        title_text = "Month",
        title_font = {"size": 20},
        title_standoff = 25)

figMonth.update_layout(legend_title = 'Year')

figMonth.update_yaxes(
        title_text = "Total Number of Rides",
        title_standoff = 25)

In [ ]:
# scattters 
# post covid correlation between commerical distance and popularity 
comscatpre = px.scatter(distinct_post[['Popularity', 'comm_dist_start', 'start_name']],
           x = 'Popularity', y='comm_dist_start', hover_name = 'start_name')

In [ ]:
comscatpost = px.scatter(distinct_post[['Popularity', 'comm_dist_end', 'start_name']],
           x = 'Popularity', y='comm_dist_end', hover_name = 'start_name')

In [ ]:
#post covid park distance and popularity correlation
px.scatter(distinct_post[['Popularity', 'park_dist_start', 'start_name']],
           x = 'Popularity', y='park_dist_start', hover_name = 'start_name')


In [ ]:
distinct_pre
distinct_post

In [ ]:
# Filtering relevant data
duration_data = df["durationmins"]
dur_fig = plt.figure(figsize=(15,7))
ax = dur_fig.add_axes([0.1, 0.1, 0.8, 0.8])

# Ploting the histogram
plt.hist(duration_data, bins = range(62),color = "blue", histtype ="bar")

# Adding median and mean lines
plt.axvline(df["durationmins"].mean(), color="MAGENTA", linestyle='-', linewidth=2 )
plt.axvline(df["durationmins"].median(), color="GREEN", linestyle='-', linewidth=2 )
plt.axvline(x = 30, color="blue", linestyle='-', linewidth=1 )

# Adding median and mean texts
min_ylim, max_ylim = plt.ylim()
plt.text(df["durationmins"].mean()*1.1, max_ylim*0.9, 'Mean: {:.0f} min'.format(df["durationmins"].mean()), color = "MAGENTA",  fontsize= 12)
plt.text(df["durationmins"].median()*1.1, max_ylim*0.8, 'Median: {:.0f} min'.format(df["durationmins"].median()), color = "GREEN", fontsize= 12)
plt.text(x= 28,y=200000, s="Free", color = "GREEN", fontsize= 12)
plt.text(x= 30.5,y=200000, s="Extra Charge", color = "grey", fontsize= 12)

# Seting ticks on x axis
ticks =range(0, 62, 2)
plt.xticks(ticks)
# Seting ticks on y axis
ticks =range(0, 7000, 2000)

# Labeling Axes
plt.xlabel("Duration of Rides", fontsize= 12)
plt.ylabel("Number of Rides", fontsize= 12)

# Adding Grid
plt.grid(linestyle=":", color='grey')

plt.show()

In [112]:
#  interactive graph 

app = dash.Dash(__name__)

    

In [113]:
# app with only map visualizations 

app.layout = html.Div([
    dcc.Textarea(
        id='Station Popularity and Expansion Maps',
        value='The maps identify stations and display station use based on color:Red >= 50k, Orange: 20k-50k Yellow: 20k-5k, Green: 5k-1k, Blue: 1000-100, Purple: < 100',
        style={'width': '100%', 'height': 50}),
    
    dcc.Tabs([ dcc.Tab(label='Pre Covid Map', children=[
                dcc.Graph(
                id='Pre-Covid Popular Regions and Expansion Areas',style={'width': '100%', 'height': '90vh'},
                figure= figpre)]),
              dcc.Tab(label='Post Covid Map', children=[
                dcc.Graph(
                id='Post-Covid Popular Regions and Expansion Areas',style={'width': '100%', 'height': '90vh'},
                figure= figpost)])
             ])
      
])
        
        
        
        
        
        
        
        
        
        
        
        

In [ ]:
# App with all visualizaitons 




# app.layout = html.Div([
#     dcc.Tabs([ dcc.Tab(label='Number of Rides', children=[
#                 dcc.Graph(
#                 id='Rides Per Month by Year',
#                 figure= figMonth), 
#                 dcc.Graph(
#                 id='Number of Rides by User Type ',
#                 figure= figRide)
#                 ]),
#             dcc.Tab(label='Station Popularity', children =[
#                 dcc.Graph(figure={"data": [
#                             {
#                                 "x": pre1["start_name"],
#                                 "y": pre1["Popularity"],
#                                 "type": "bar",
#                             },
#                         ],
#                         "layout": {"title": "Pre Covid Station Popularity"},
#                     },
#                         ),
#                 dcc.Graph( 
#                    figure={ "data": [{"x": post1["start_name"],"y": post1["Popularity"], "type": "bar"}],
#                            "layout": {"title": "Post Covid Station Popularity"},
#                     }
                    
#                     )]),
            
#             dcc.Tab(label='Pre Covid Map', children=[
#                 dcc.Graph(
#                 id='Pre-Covid Popular Regions and Expansion Areas',style={'width': '90vh', 'height': '90vh'},
#                 figure= figpre),
#                 dcc.Graph(
#                 id='Post-Covid Popular Regions and Expansion Areas',style={'width': '90vh', 'height': '90vh'},
#                 figure= figpost)
#             ]),
#              dcc.Tab(label='Correlations', children=[
#                 dcc.Graph(
#                 id='Correlation between commerical distance start and station popularity',
#                 figure=comscatpre),
#                 dcc.Graph(
#                 id='Correlation between commercial distance end and station poularity',
#                 figure=comscatpost )
#              ])
#              ])
#              ])

               
             
        


In [114]:
if __name__ == '__main__':
    app.run_server(debug=False)

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:8050/ (Press CTRL+C to quit)
127.0.0.1 - - [30/Mar/2022 20:09:07] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [30/Mar/2022 20:09:07] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [30/Mar/2022 20:09:07] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [30/Mar/2022 20:09:07] "GET /_dash-component-suites/dash/dcc/async-graph.js HTTP/1.1" 200 -
127.0.0.1 - - [30/Mar/2022 20:09:07] "GET /_dash-component-suites/dash/dcc/async-plotlyjs.js HTTP/1.1" 200 -
